# $\S$ 7.10. Cross-Validation
## $\S$ 7.10.1. $K$-Fold Cross-Validation

Ideally, if we had enough data, we would set aside a validation set and use it to assess the performance of our prediction model. Since data are often scarce, this is usually not possible. To finesse the problem, $K$-fold cross-validation uses part of the available data to fit the model, and a different part to test it.

We split the data into $K$ roughly equal-sized parts; for example, when $K=5$, the scenario looks likes this:

 1 | 2 | 3 | 4 | 5
---|---|---|---|---
Train|Train|Validation|Train|Train

For the $k$th part (third above), we fit the model to the other $K-1$ parts of the data, and calculate the prediction error of the fitted model when predicting the $k$th part of the data. We do this for $k=1,2,\cdots,K$ and combine the $K$ estimates of prediction error.

Here are more details. Let
* $\kappa: \{1,\cdots,N\} \mapsto \{1,\cdots,K\}$ be an indexing function that indicates the partition to which observation $i$ is allocated by the randomization.
* $\hat{f}^{-k}(x)$ be the fitted function, computed with the $k$th part of the data removed.

Then the cross-validation estimate of prediction error is

\begin{equation}
\text{CV}(\hat{f}) = \frac{1}{N} \sum_{i=1}^N L\left( y_i, \hat{f}^{-\kappa(i)}(x_i) \right).
\end{equation}

Typically choices of $K$ are 5 or 10 (see below). The case $K=N$ is known as _leave-one-out_ cross-validation. In this case $\kappa(i) = i$, and for $i$th observation the fit is computed using all the data except the $i$th.

### With tuning parameters

* Given a set of models $f(x,\alpha)$ indexed by a tuning parameter $\alpha$,
* denote by $\hat{f}^{-k}(x, \alpha)$ the $\alpha$th model fit with the $k$the part of the data removed.

Then for this set of models we define

\begin{equation}
\text{CV}(\hat{f}, \alpha) = \frac{1}{N} \sum_{i=1}^N L(y_i, \hat{f}^{-\kappa(i)}(x_i,\alpha)).
\end{equation}

The function $\text{CV}(\hat{f}, \alpha)$ provides an estimate of the test error curve, and we find the tuning parameter $\hat\alpha$ that minimizes it. Our final chosen model is $f(x,\hat\alpha)$, which we then fit to all the data.

### What $K$-fold CV estimates

It is interesting to wonder about what quantity $K$-fold cross-validation estimates.

With $K=5$ or $10$, we might guess that it estimates the expected error $\text{Err}$, since the training sets in each fold are quite different from the original training set.

On the other hand, if $K=N$ we might guess that cross-validation estimates the conditional error $\text{Err}_{\mathcal{T}}$. It turns out that cross-validation only estimates effectively the average error $\text{Err}$, as discussed in $\S$ 7.12.

### How to choose $K$

What value should we choose for $K$?

With $K=N$, the cross-validaton estimator is approximately unbiased for the true (expected) prediction error, but can have high variance because the $N$ "training sets" are so similar to one another. The computational burden is also considerable, requiring $N$ applications of the learning method. In certain problems, this computation can be done quickly -- see Exercise 7.3 and 5.13.

On the other hand, with $K=5$ say, cross-validation has lower variance. But bias could be a problem, depending on how the performance of the learning method varies with the size of the training set.

FIGURE 7.8 shows a hypothetical "learning curve" for a classifier on a given task, a plot of $1-\text{Err}$ versus the size of the training set $N$. The performance of the classifier improves as the training set size increases to 100 observations; increasing the number further to 200 brings only a small benefit.
* If our training set had 200 observation, fivefold cross-validation would estimate the performance  of our classifier over training sets of size 160, which from FIGURE 7.8 is virtually the same as the performance for training set size 200. Thus cross-validation would not suffer from much bias.
* However if the training set had 50 observations, fivefold cross-validation would estimate the performace of our classifier over training sets of size 40, and from the figure that would be an underestimate of $1-\text{Err}$. Hence as an estimate of $\text{Err}$, cross-validation would be biased upward.

To summarize, of the learning curve has a considerable slope at the given training set size, 5- or 10-fold cross-validation will overestimate the true prediction error. Whether this bias is a drawback in practice depends on the objective.

On the other hand, leave-one-out cross-validation has low bias but can have high variance. 

Overall, 5- or 10-fold cross-validation are recommended as a good compromise: see Breiman and Spector (1992) and Kohavi (1995).

FIGURE 7.9 shows the prediction error and tenfold cross-validation curve estimated from a single training set, from the scenario in the bottom right panel of FIGURE 7.3.

This is a two-class classification problem, using a linear model with best subsets regression of subset size $p$. Standard error bars are shown, which are the standard errors of the individual misclassification error rates for each of the ten parts. Both curves have minima at $p=10$, although the CV curve is rather flat beyond 10.

#### One-standard error rule
Often a "one-standard error" rule is used with cross-validation, in which we choose the most parsimonious model whose error is no more than one standard error above the error of the best model.

Here it looks like a model with about $p=9$ predictors would be chosen, while the true model uses $p=10$.

### Generalized cross-validation

_Generalized cross-validation_ provides a convenient approximation to leave-one out cross-validation, for linear fitting under squared-error loss.

As defined in $\S$ 7.6, a linear fitting method is one for which we can write

\begin{equation}
\hat{\mathbf{y}} = \mathbf{Sy}.
\end{equation}

Now for many linear fitting methods,

\begin{equation}
\frac{1}{N} \sum_{i=1}^N \left[ y_i - \hat{f}^{-i}(x_i) \right]^2 = \frac{1}{N} \sum_{i=1}^N \left[ \frac{y_i - \hat{f}(x_i)}{1-S_{ii}} \right]^2,
\end{equation}

where $S_{ii}$ is the $i$th diagonal element of $\mathbf{S}$ (see Exercise 7.3).

The GCV approximation is

\begin{equation}
\text{GCV}(\hat{f}) = \frac{1}{N} \sum_{i=1}^N \left[ \frac{y_i - \hat{f}(x_i)}{1-\text{trace}(\mathbf{S})/N} \right]^2.
\end{equation}

The quantity $\text{trace}(\mathbf{S})$ is the effective number of parameters, as defined in $\S$ 7.6.

GCV can have a computational advantage in some settings, where the trace of $\mathbf{S}$ can be computed more easily than the individual elements $S_{ii}$.

In smoothing problems, GCV can also alleviate the tendency of cross-validation to undersmooth. The similarity between GCV and AIC can be seen from the approximation (Exercise 7.7)

\begin{equation}
\frac{1}{(1-x)^2} \approx 1 + 2x.
\end{equation}